In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
hits_train = pd.read_csv("train.csv", index_col='global_id')
hits_test = pd.read_csv("test.csv", index_col='global_id')

In [7]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.cross_validation import cross_val_score

In [11]:
classifier = RandomForestClassifier(criterion='gini', n_jobs=-1)
cv_gini = cross_val_score(classifier,
                hits_train[['energy_deposit', 'relative_time']].values, (hits_train.label == 1).values.astype(np.int),
               scoring='roc_auc')
print(cv_gini.mean(), cv_gini.std())

0.987512733727 0.00207648248863


In [14]:
classifier.fit(hits_train[['energy_deposit', 'relative_time']], (hits_train.label == 1))

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [15]:
ml_prediction = pd.DataFrame({
        "prediction": classifier.predict_proba(hits_test[[
                    'energy_deposit', 'relative_time']])[:, 1]}, index=hits_test.index)

In [19]:
hits_test_is_signal = (hits_test.label == 1)
score = roc_auc_score(hits_test_is_signal, ml_prediction)

In [20]:
with open("ml_score.dat", "w") as score_io:
    score_io.write(str(score))